In [1]:
from set_up import *
from pybreaks.adjust_freq_quantile_matching import QuantileCatMatch
pd.options.display.max_rows=7
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# this cell loads either test data with properties as set by the user or real data and allows to introduce a break
def load_data(real=False, trend=0.01, bias=0.1, breaktime=datetime(2012,7,1), 
              size=(0.2, 1.1), resample=False):
    if real:
        CAN, REF = load_real_data(707393, breaktime, size, resample)
    else:
        CAN, REF = load_test_data(trend, bias, breaktime, size, resample, seed=1234)
    return CAN, REF

In [3]:
@interact(TREND=(0, 0.05, 0.01), BIAS=(-0.5, 0.5, 0.1), BREAK_ADD=(-0.4, 0.4, 0.05), 
          BREAK_MULT=(0.7, 1.3, 0.05), QUANTILE_CATEGORIES=(1,12,1), 
          RESAMPLE=['NONE', 'INPUT', 'OUTPUT'])
def QCM(USE_REAL_DATA=False, TESTDATA_TREND=0.01, TESTDATA_BIAS=0.1, BREAK_ADD=0.2, 
        BREAK_MULT=1.1, QUANTILE_CATEGORIES=4, RESAMPLE='NONE'):
    
    BREAKTIME=datetime(2016,12,31)
    size=(BREAK_ADD, BREAK_MULT)
    
    resample = True if RESAMPLE=='INPUT' else False
    CAN, REF = load_data(USE_REAL_DATA, TESTDATA_TREND, TESTDATA_BIAS, BREAKTIME, 
                         size, resample)
    
    qcm = QuantileCatMatch(CAN, REF, BREAKTIME, bias_corr_method='cdf_match', adjust_group=0, 
                           n_quantiles=QUANTILE_CATEGORIES, 
                           first_last='formula', fit='mean')
    
    can_adjusted = qcm.adjust(CAN[:BREAKTIME], interpolation_method='cubic')

    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,4)) 
    if RESAMPLE == 'OUTPUT':
        qcm.df_original.resample('M').mean().plot(ax=ax1)
    else:
        qcm.df_original.plot(ax=ax1)
    
    qcm.plot_cdf_compare(ax=ax2)

    qcm.plot_models()
    
        


aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgRmxvYXRTbGlkZXIodmFsdWU9MC4wMSwgZGVzY3JpcHRpb27igKY=
